In [ ]:
import requests
import urllib3
from datetime import datetime

# SSL 경고 비활성화
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# API 기본 URL 및 데이터 이름 설정
url = "https://www.safetydata.go.kr"  # API 기본 URL
dataName = "/V2/api/DSSP-IF-00247?serviceKey=N930APQ7S44G4I9I"
servicekey = "N930APQ7S44G4I9I"
total_data = []  # 전체 데이터를 저장할 리스트
unique_sn = set()  # 중복 일련번호를 저장할 집합

numOfRows = 10  # 한 번에 요청할 데이터 수
pageNo = 1  # 페이지 번호 초기화

# 오늘 날짜를 'YYYYMMDD' 형식으로 가져오기
current_date = datetime(2024, 12, 2).strftime('%Y%m%d')  # 특정 날짜로 설정

# 사용자로부터 원하는 지역 입력 받기
desired_region = input("알림을 받고 싶은 지역을 입력하세요: ")

while True:
    payloads = {
        "servicekey": servicekey,
        "returnType": "json",
        "pageNo": str(pageNo),
        "numOfRows": str(numOfRows),
        "crtDt": current_date  # 특정 날짜를 조회 시작일자로 설정
    }

    # API 요청
    response = requests.get(url + dataName, params=payloads)

    if response.status_code == 200:
        data = response.json()

        # 'body' 키가 존재하는지 확인
        if 'body' in data and data['body'] is not None:
            # 각 항목에서 원하는 지역의 알림만 필터링
            for item in data['body']:
                if item.get('RCPTN_RGN_NM') == desired_region:  # 지역 필터링
                    sn = item.get('SN')  # 일련번호 가져오기
                    if sn not in unique_sn:  # 일련번호가 집합에 없으면
                        unique_sn.add(sn)  # 집합에 추가
                        # 불필요한 필드를 제외한 새로운 딕셔너리 생성
                        filtered_item = {
                            'MSG_CN': item.get('MSG_CN'),
                            'RCPTN_RGN_NM': item.get('RCPTN_RGN_NM'),
                            'CRT_DT': item.get('CRT_DT'),
                            'SN': sn,
                            'DST_SE_NM': item.get('DST_SE_NM')
                        }
                        total_data.append(filtered_item)  # 필터링된 아이템 추가

            # 다음 페이지가 없으면 종료
            if len(data['body']) < numOfRows:
                break

            pageNo += 1  # 다음 페이지로 이동
        else:
            print("응답에 데이터가 없습니다:", data)
            break
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        break

# 전체 데이터 출력 (필터링된 지역의 알림만)
if total_data:
    print(f"{desired_region}의 알림 메시지:")
    for item in total_data:
        print(item)  # 필터링된 데이터 출력
else:
    print(f"{desired_region}에 대한 알림이 없습니다.")


{'MSG_CN': '밤사이 눈이 얼어 보도 등 도로 곳곳이 많이 미끄럽습니다. 보행에 주의하시고 특히 어린이, 노약자 등은 이동시 안전사고에 유의하시기 바랍니다. [시흥시청]', 'RCPTN_RGN_NM': '경기도 시흥시 ', 'CRT_DT': '2024/11/30 07:55:04', 'SN': 227263, 'DST_SE_NM': '대설'}
{'MSG_CN': '오늘 08:59경 수성구 황금동 368, 캐슬골드파크 4단지 화재 발생, 인근 주민은 안전 사고 발생에 유의바랍니다. [수성구청]', 'RCPTN_RGN_NM': '대구광역시 수성구 범어동,대구광역시 수성구 만촌동,대구광역시 수성구 수성동1가,대구광역시 수성구 수성동2가,대구광역시 수성구 수성동3가,대구광역시 수성구 수성동4가,대구광역시 수성구 중동,대구광역시 수성구 상동,대구광역시 수성구 파동,대구광역시 수성구 두산동,대구광역시 수성구 지산동,대구광역시 수성구 범물동,대구광역시 수성구 시지동,대구광역시 수성구 매호동,대구광역시 수성구 성동,대구광역시 수성구 사월동,대구광역시 수성구 신매동,대구광역시 수성구 욱수동,대구광역시 수성구 노변동,대구광역시 수성구 삼덕동,대구광역시 수성구 연호동,대구광역시 수성구 이천동,대구광역시 수성구 고모동,대구광역시 수성구 가천동,대구광역시 수성구 대흥동', 'CRT_DT': '2024/12/02 09:48:17', 'SN': 227280, 'DST_SE_NM': '화재'}
{'MSG_CN': '오늘 21시부터 인천지역 한파주의보 발효로 인한 강추위가 예상됩니다. 야외활동 자제, 외출시 방한용품 착용, 수도계량기 동파방지 등 유의 바랍니다.[인천광역시]', 'RCPTN_RGN_NM': '인천광역시 전체 ', 'CRT_DT': '2024/12/02 17:15:41', 'SN': 227282, 'DST_SE_NM': '한파'}
{'MSG_CN': '밤사이 기온 강하로 도로 결빙에 인한 교통사고 우려가 있습니다. 감속운행, 차간 안전거리 유지 등 교통안전에 유의하시기 바

In [5]:
import pandas as pd

# 예시로, 총 10개의 데이터에 대해 인명피해 발생 여부를 레이블링합니다.
# 실제로는 여러분의 데이터에 맞춰 레이블을 설정해야 합니다.
labels = [1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
          1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
          1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
          1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
          1, 0]  # 1: 인명피해 발생, 0: 인명피해 미발생 (예시)

# 데이터프레임으로 변환
df = pd.DataFrame(total_data)
df['label'] = labels  # 레이블 추가

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

# KoBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2)

# 데이터 토크나이징
inputs = tokenizer(df['MSG_CN'].tolist(), padding=True, truncation=True, return_tensors="pt", max_length=512)
labels = torch.tensor(df['label'].tolist())

# 데이터셋 만들기
class DisasterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

dataset = DisasterDataset(inputs, labels)

# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir='./results',          # 출력 디렉토리
    num_train_epochs=3,              # 학습할 에폭 수
    per_device_train_batch_size=8,   # 각 디바이스당 배치 크기
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',            # 로그 디렉토리
)

# Trainer 객체 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

# 모델 학습
trainer.train()


C:\Users\김남현\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 18/18 [01:00<00:00,  3.35s/it]

{'train_runtime': 60.2745, 'train_samples_per_second': 2.09, 'train_steps_per_second': 0.299, 'train_loss': 0.7189866701761881, 'epoch': 3.0}


TrainOutput(global_step=18, training_loss=0.7189866701761881, metrics={'train_runtime': 60.2745, 'train_samples_per_second': 2.09, 'train_steps_per_second': 0.299, 'total_flos': 2007249574680.0, 'train_loss': 0.7189866701761881, 'epoch': 3.0})

In [ ]:
# 평가 데이터프레임 생성
test_df = pd.DataFrame(test_data)

# 데이터 토크나이징
test_inputs = tokenizer(test_df['MSG_CN'].tolist(), padding=True, truncation=True, return_tensors="pt", max_length=512)
test_labels = torch.tensor(test_df['label'].tolist())

# 데이터셋 만들기
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = TestDataset(test_inputs, test_labels)


In [ ]:
test_data = [
{'MSG_CN': '오늘 08:59경 수성구 황금동 368, 캐슬골드파크 4단지 화재 발생, 인근 주민은 안전 사고 발생에 유의바랍니다. [수성구청]', 'RCPTN_RGN_NM': '대구광역시 수성구 범어동,대구광역시 수성구 만촌동,대구광역시 수성구 수성동1가,대구광역시 수성구 수성동2가,대구광역시 수성구 수성동3가,대구광역시 수성구 수성동4가,대구광역시 수성구 중동,대구광역시 수성구 상동,대구광역시 수성구 파동,대구광역시 수성구 두산동,대구광역시 수성구 지산동,대구광역시 수성구 범물동,대구광역시 수성구 시지동,대구광역시 수성구 매호동,대구광역시 수성구 성동,대구광역시 수성구 사월동,대구광역시 수성구 신매동,대구광역시 수성구 욱수동,대구광역시 수성구 노변동,대구광역시 수성구 삼덕동,대구광역시 수성구 연호동,대구광역시 수성구 이천동,대구광역시 수성구 고모동,대구광역시 수성구 가천동,대구광역시 수성구 대흥동', 'CRT_DT': '2024/12/02 09:48:17', 'SN': 227280, 'DST_SE_NM': '화재', "label": 1},
{'MSG_CN': '이면 도로, 보도 결빙으로 안전사고의 위험이 있습니다. 불필요한 외출 및 여행은 가급적 삼가시고 부득이 외출시 각별히 주의하시기 바랍니다. [시흥시청]', 'RCPTN_RGN_NM': '경기도 시흥시 ', 'CRT_DT': '2024/12/01 07:00:17', 'SN': 227273, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '밤사이 일부 도로의 결빙(블랙아이스)으로 교통사고 우려가 있으니 차량운행시 안전운전에 유의하시고, 결빙지역 보행시 미끄럼 사고에 주의하시기 바랍니다.[경기도청]', 'RCPTN_RGN_NM': '경기도 전체 ', 'CRT_DT': '2024/12/01 07:20:08', 'SN': 227274, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '도내 전 지역 영상의 기온이며 어제에 이어 제설 작업 진행 중입니다. 가족과 이웃의 안전을 위해 내 집 내 점포 앞 눈 쓸기에 동참해 주시기 바랍니다 [경기도청]', 'RCPTN_RGN_NM': '경기도 전체 ', 'CRT_DT': '2024/12/01 14:26:44', 'SN': 227277, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '오늘 08:59경 수성구 황금동 368, 캐슬골드파크 4단지 화재 발생, 인근 주민은 안전 사고 발생에 유의바랍니다. [수성구청]', 'RCPTN_RGN_NM': '대구광역시 수성구 황금동', 'CRT_DT': '2024/12/02 09:37:30', 'SN': 227279, 'DST_SE_NM': '화재', "label": 1},
{'MSG_CN': '이면도로 고가차도 등 결빙으로 인한 교통사고의 우려가 있으니 안전거리 확보 등 교통안전에 유의하시고 보행 시 미끄럼사고에 주의하여 주시기 바랍니다.[안산시청]', 'RCPTN_RGN_NM': '경기도 안산시 상록구 ,경기도 안산시 단원구 ', 'CRT_DT': '2024/12/01 07:31:38', 'SN': 227275, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '폭설로 인해 떨어진 전선은 감전위험이 있으니 절대 만지거나 접촉하지 마시고 안전에 특별히 유의하여 주시기 바랍니다.[양평군청]', 'RCPTN_RGN_NM': '경기도 양평군 ', 'CRT_DT': '2024/12/01 15:17:44', 'SN': 227278, 'DST_SE_NM': '정전', "label": 0},
{'MSG_CN': '양평군 지역에 짙은 안개 발생. 안전거리를 확보하고 빙판길이나 도로살얼음이 나타날 수 있으니 교통 및 보행안전에 주의하시기 바랍니다.[양평군청]', 'RCPTN_RGN_NM': '경기도 양평군 ', 'CRT_DT': '2024/12/01 05:52:55', 'SN': 227272, 'DST_SE_NM': '안개', "label": 0},
{'MSG_CN': '폭설로 인해 떨어진 전선은 감전위험이 있으니 절대 만지거나 접촉하지 마시고 안전에 특별히 유의하여 주시기 바랍니다.[양평군청]', 'RCPTN_RGN_NM': '경기도 양평군 ', 'CRT_DT': '2024/12/02 15:40:12', 'SN': 227281, 'DST_SE_NM': '정전', "label": 0},
{'MSG_CN': '밤사이 일부 도로의 결빙(블랙아이스)으로 교통사고 우려가 있으니 차량운행시 안전운전에 유의하시고, 결빙지역 보행시 미끄럼 사고에 주의하시기 바랍니다.[광주시]', 'RCPTN_RGN_NM': '경기도 광주시 ', 'CRT_DT': '2024/12/01 09:13:23', 'SN': 227276, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '밤사이 눈이 얼어 보도 등 도로 곳곳이 많이 미끄럽습니다. 보행에 주의하시고 특히 어린이, 노약자 등은 이동시 안전사고에 유의하시기 바랍니다. [시흥시청]', 'RCPTN_RGN_NM': '경기도 시흥시 ', 'CRT_DT': '2024/11/30 07:55:04', 'SN': 227263, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '오늘 09:30경 관로누수로 인하여 문산 일부지역 단수중이며 11시경 복구 예정입니다. 식수 및 생활용수 확보 등 단수에 유의바랍니다.[파주시]', 'RCPTN_RGN_NM': '경기도 파주시 문산읍', 'CRT_DT': '2024/11/30 09:56:44', 'SN': 227267, 'DST_SE_NM': '수도', "label": 0},
{'MSG_CN': '이면 도로, 보도 결빙으로 안전사고의 위험이 있습니다. 불필요한 외출 및 여행은 가급적 삼가시고 부득이 외출시 각별히 주의하시기 바랍니다. [시흥시청]', 'RCPTN_RGN_NM': '경기도 시흥시 ', 'CRT_DT': '2024/12/01 07:00:17', 'SN': 227273, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '동서로 둔대고가도로 결빙으로, 물왕에서 월곶 방향 진입차량은 우회하여 주시기 바랍니다. [시흥시청]', 'RCPTN_RGN_NM': '경기도 시흥시 도창동,경기도 시흥시 금이동,경기도 시흥시 능곡동,경기도 시흥시 하중동,경기도 시흥시 하상동,경기도 시흥시 광석동,경기도 시흥시 물왕동,경기도 시흥시 산현동,경기도 시흥시 군자동,경기도 시흥시 장현동,경기도 시흥시 장곡동', 'CRT_DT': '2024/11/30 08:12:10', 'SN': 227264, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '이면 도로, 보도 결빙으로 안전사고의 위험이 있을 수 있습니다. 불필요한 외출 및 여행은 가급적 삼가시고 부득이 외출 시 각별히 주의하시기 바랍니다.[광주시]', 'RCPTN_RGN_NM': '경기도 광주시 ', 'CRT_DT': '2024/11/30 08:14:42', 'SN': 227265, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '내린 눈이 얼어 미끄럼 사고가 우려됩니다. 고갯길, 교량, 터널 앞 결빙구간에서 감속 운전과 차간 거리 유지하여 주시고, 보행 시 낙상사고에 유의하세요[화성시]', 'RCPTN_RGN_NM': '경기도 화성시 ', 'CRT_DT': '2024/11/30 17:36:31', 'SN': 227271, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '그간 내린 다량의 습설로 인해 비닐하우스, 축사 등 적설취약구조물에 쌓인 눈으로 붕괴 위험이 예상될 경우, 사전대피 및 제설작업은 삼가주시기 바랍니다[충청북도]', 'RCPTN_RGN_NM': '충청북도 음성군 ,충청북도 진천군 ,충청북도 충주시 ', 'CRT_DT': '2024/11/30 12:16:35', 'SN': 227269, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '27일~28일간 적설된 눈에 의한 노후시설물 등의 붕괴가 발생할 수 있으니, 주민께서는 안전사고가 발생하지 않도록 각별히 주의하시기 바랍니다.[음성군]', 'RCPTN_RGN_NM': '충청북도 음성군 ', 'CRT_DT': '2024/11/30 09:04:49', 'SN': 227266, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '밤사이 일부 도로의 결빙(블랙아이스)으로 교통사고 우려가 있으니 차량운행시 안전운전에 유의하시고, 결빙지역 보행시 미끄럼 사고에 주의하시기 바랍니다.[경기도청]', 'RCPTN_RGN_NM': '경기도 전체 ', 'CRT_DT': '2024/12/01 07:20:08', 'SN': 227274, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '기온 하강으로 인한 결빙 및 블랙아이스(노면 살얼음)로 인하여 교통사고가 우려되오니 교량, 고가도로에서 서행하시고 교통안전에 유의하시기 바랍니다.[김제시]', 'RCPTN_RGN_NM': '전북특별자치도 김제시 ', 'CRT_DT': '2024/11/30 07:46:06', 'SN': 227262, 'DST_SE_NM': '한파', "label": 1},
{'MSG_CN': '도내 대부분 지역이 영상의 기온이며 주말 제설 작업은 계속 진행될 예정입니다. 가족과 이웃의 안전을 위해 내집 내점포앞 눈 쓸기에 동참해주시기 바랍니다[경기도청]', 'RCPTN_RGN_NM': '경기도 전체 ', 'CRT_DT': '2024/11/30 10:30:46', 'SN': 227268, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '도내 전 지역 영상의 기온이며 어제에 이어 제설 작업 진행 중입니다. 가족과 이웃의 안전을 위해 내 집 내 점포 앞 눈 쓸기에 동참해 주시기 바랍니다 [경기도청]', 'RCPTN_RGN_NM': '경기도 전체 ', 'CRT_DT': '2024/12/01 14:26:44', 'SN': 227277, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '이면 도로, 보도 결빙으로 안전사고의 위험이 있습니다. 불필요한 외출 및 여행은 가급적 삼가시고 부득이 외출 시 각별히 주의하시기 바랍니다. [경기도청]', 'RCPTN_RGN_NM': '경기도 전체 ', 'CRT_DT': '2024/11/30 07:00:09', 'SN': 227254, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '어제 내린 비와 영하의 기온으로 도로 결빙 우려 ▲서행운전 ▲안전거리 확보 ▲교량, 터널 앞, 고가도로 결빙 위험구간 주의 등 안전운전에 유의하세요. [세종시청]', 'RCPTN_RGN_NM': '세종특별자치시  ', 'CRT_DT': '2024/11/30 07:00:19', 'SN': 227255, 'DST_SE_NM': '기타', "label": 1},
{'MSG_CN': '기온 하강으로 인한 결빙 및 블랙아이스(노면 살얼음)로 인하여 교통사고가 우려되오니 교량, 고가도로에서 서행하시고 교통안전에 유의하시기 바랍니다. [임실군]', 'RCPTN_RGN_NM': '전북특별자치도 임실군 ', 'CRT_DT': '2024/11/30 07:06:07', 'SN': 227256, 'DST_SE_NM': '한파', "label": 1},
{'MSG_CN': '기온이 떨어져 도로결빙에 의한 사고가 우려되오니, 시민여러분께서는 운전 시 감속 운행 및 미끄럼 사고에 주의바랍니다.[전주시]', 'RCPTN_RGN_NM': '전북특별자치도 전주시 완산구 ,전북특별자치도 전주시 덕진구 ', 'CRT_DT': '2024/11/30 07:13:41', 'SN': 227257, 'DST_SE_NM': '한파', "label": 1},
{'MSG_CN': '기온 하강으로 인한 결빙 및 블랙아이스(노면 살얼음)로 인하여 교통사고가 우려되오니 교량, 고가도로 등에서 서행하시고 교통안전에 유의하시기 바랍니다. [군산시]', 'RCPTN_RGN_NM': '전북특별자치도 군산시 ', 'CRT_DT': '2024/11/30 07:15:21', 'SN': 227258, 'DST_SE_NM': '한파', "label": 1},
{'MSG_CN': '이면도로 고가차도 등 결빙으로 인한 교통사고의 우려가 있으니 안전거리 확보 등 교통안전에 유의하시고 보행 시 미끄럼사고에 주의하여 주시기 바랍니다.[안산시청]', 'RCPTN_RGN_NM': '경기도 안산시 상록구 ,경기도 안산시 단원구 ', 'CRT_DT': '2024/12/01 07:31:38', 'SN': 227275, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '기온 하강으로 인한 결빙 및 블랙아이스(노면 살얼음)로 인하여 교통사고가 우려되오니 교량, 고가도로에서 서행하시고 교통안전에 유의하시기 바랍니다. [완주군]', 'RCPTN_RGN_NM': '전북특별자치도 완주군 ', 'CRT_DT': '2024/11/30 06:35:20', 'SN': 227251, 'DST_SE_NM': '한파', "label": 1},
{'MSG_CN': '청양군 청양읍 벽천리 569-3번지(여주재터널방향, 36번국도) 빙판길로 인한 차량 사고 발생 ▲빙판길 주의▲서행운전 등 유의하시기 바랍니다.[청양군]', 'RCPTN_RGN_NM': '충청남도 청양군 ', 'CRT_DT': '2024/11/30 07:19:26', 'SN': 227259, 'DST_SE_NM': '기타', "label": 0},
{'MSG_CN': '많은눈이 내려 쌓여있고, 기온이 낮아지면서 빙판길과 도로 살얼음이 나타나는곳이 많습니다. 교통안전과 보행자 낙상사고에 각별히 주의하여 주시기 바랍니다. [양평군]', 'RCPTN_RGN_NM': '경기도 양평군 ', 'CRT_DT': '2024/11/30 07:23:40', 'SN': 227260, 'DST_SE_NM': '한파', "label": 1},
{'MSG_CN': '기온 하강으로 인한 결빙 및 블랙아이스(노면 살얼음)로 인하여 교통사고가 우려되오니 교량, 고가도로에서 서행하시고 교통안전에 유의하시기 바랍니다. [익산시]', 'RCPTN_RGN_NM': '전북특별자치도 익산시 ', 'CRT_DT': '2024/11/30 07:23:50', 'SN': 227261, 'DST_SE_NM': '한파', "label": 1},
{'MSG_CN': '폭설로 인해 떨어진 전선은 감전위험이 있으니 절대 만지거나 접촉하지 마시고 안전에 특별히 유의하여 주시기 바랍니다.[양평군청]', 'RCPTN_RGN_NM': '경기도 양평군 ', 'CRT_DT': '2024/12/02 15:40:12', 'SN': 227281, 'DST_SE_NM': '정전', "label": 0},
{'MSG_CN': '밤사이 일부 도로의 결빙(블랙아이스)으로 교통사고 우려가 있으니 차량운행시 안전운전에 유의하시고, 결빙지역 보행시 미끄럼 사고에 주의하시기 바랍니다.[광주시]', 'RCPTN_RGN_NM': '경기도 광주시 ', 'CRT_DT': '2024/12/01 09:13:23', 'SN': 227276, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '폭설로 인해 떨어진 전선은 감전위험이 있으니 절대 만지거나 접촉하지 마시고 안전에 특별히 유의하여 주시기 바랍니다. [양평군]', 'RCPTN_RGN_NM': '경기도 양평군 ', 'CRT_DT': '2024/11/30 14:53:20', 'SN': 227270, 'DST_SE_NM': '대설', "label": 1},
{'MSG_CN': '기온 하강으로 인한 결빙 및 블랙아이스(노면 살얼음)로 인하여 교통사고가 우려되오니 교량, 고가도로에서 서행하시고 교통안전에 유의하시기 바랍니다. [장수군]', 'RCPTN_RGN_NM': '전북특별자치도 장수군 ', 'CRT_DT': '2024/11/30 06:39:41', 'SN': 227252, 'DST_SE_NM': '한파', "label": 1},
{'MSG_CN': '기온하강으로 인한 결빙 및 블랙아이스(노면 살얼음)가 우려되오니 고갯길, 교량, 고가도로에서 서행하시고 교통 안전에 유의하시기 바랍니다.[정읍시]', 'RCPTN_RGN_NM': '전북특별자치도 정읍시 ', 'CRT_DT': '2024/11/30 06:41:40', 'SN': 227253, 'DST_SE_NM': '한파', "label": 1},
{'MSG_CN': '[종로구] 4월 28일 21:05 지진발생/추가 지진 발생상황에 유의 바람-종로구', 'DST_SE_NM': '지진', "label": 1}
]

In [ ]:
# 평가
trainer = Trainer(
    model=model,
)

eval_results = trainer.evaluate(test_dataset)

# 평가 결과 출력
print(eval_results)


In [ ]:
# 새로운 데이터 예측
new_data = ["오늘 08:59경 수성구 황금동 368, 캐슬골드파크 4단지 화재 발생, 인근 주민은 안전 사고 발생에 유의바랍니다. [수성구청]"]

new_inputs = tokenizer(new_data, padding=True, truncation=True, return_tensors="pt", max_length=512)

with torch.no_grad():
    outputs = model(**new_inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

print(predictions)  # 예측 결과 출력

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score

# KoBERT 모델과 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=2)

# 테스트 데이터 (예시)
test_data = [
    {"MSG_CN": "오늘 08:59경 수성구 황금동 368, 캐슬골드파크 4단지 화재 발생, 인근 주민은 안전 사고 발생에 유의바랍니다. [수성구청]", "label": 1},
    {"MSG_CN": "이면 도로, 보도 결빙으로 안전사고의 위험이 있습니다. 불필요한 외출 및 여행은 가급적 삼가시고 부득이 외출시 각별히 주의하시기 바랍니다. [시흥시청]", "label": 1},
    # 추가 테스트 데이터...
]

# 예측을 위한 입력 데이터 준비
test_messages = [item['MSG_CN'] for item in test_data]
actual_labels = [item['label'] for item in test_data]

# 데이터 토크나이징
test_inputs = tokenizer(test_messages, padding=True, truncation=True, return_tensors="pt", max_length=512)

# 예측 수행
with torch.no_grad():
    outputs = model(**test_inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# 정확도 계산
accuracy = accuracy_score(actual_labels, predictions.numpy())
print(f'Accuracy: {accuracy:.4f}')  # 정확도 출력